This demonstrates all the steps in my candidate selection before conducting visual inspection

In [1]:
import numpy as np
import splat
import wisps.data_analysis as wispd
from wisps.data_analysis import selection_criteria as sel_crt
import wispshapes
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import wisps

import matplotlib as mpl

#matplotlib defaults
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linewidth'] = 0.1
mpl.rcParams['figure.figsize'] = [6.0, 3.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 18
mpl.rcParams['legend.fontsize'] = 'small'
mpl.rcParams['figure.titlesize'] = 'large'

import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#uncomment to create index-index spaces again
dt=wisps.datasets['rf_classified']

In [3]:
#p=plt.plot(dt[wisps.INDEX_NAMES].astype(float).replace(np.nan, -9999).replace(0.0, -9999))

In [4]:
sel_crt.save_criteria(conts=dt)

  0%|          | 0/45 [00:00<?, ?it/s]

                                             x  \
4   (0.9024016455283143, 0.016422706107656063)   
9    (0.9435587168251433, 0.00767355624927435)   
10  (0.9400011914500676, 0.008424370812232315)   
12    (0.9149521764134011, 0.0164637589575767)   
15  (0.9619608194939945, 0.019559425790253146)   
20   (0.8404246672685162, 0.01702721652907841)   

                                              y  
4     (0.5769129551564539, 0.02971718733175986)  
9   (0.5105108500116876, 0.0077771662575367925)  
10   (0.5553529103933509, 0.018526231646151248)  
12   (0.5012076403090847, 0.015131894697667991)  
15    (0.504211458361207, 0.028603920488042268)  
20      (0.421604658468865, 0.0152388613193526)  
[[0.90240165 0.94355872 0.94000119 0.91495218 0.96196082 0.84042467]
 [0.57691296 0.51051085 0.55535291 0.50120764 0.50421146 0.42160466]
 [0.01642271 0.00767356 0.00842437 0.01646376 0.01955943 0.01702722]
 [0.02971719 0.00777717 0.01852623 0.01513189 0.02860392 0.01523886]] M7-L0
                

  2%|▏         | 1/45 [00:01<01:23,  1.90s/it]

                                             x  \
4   (0.9024016455283143, 0.016422706107656063)   
9    (0.9435587168251433, 0.00767355624927435)   
10  (0.9400011914500676, 0.008424370812232315)   
12    (0.9149521764134011, 0.0164637589575767)   
15  (0.9619608194939945, 0.019559425790253146)   
20   (0.8404246672685162, 0.01702721652907841)   

                                              y  
4    (0.6815742431331088, 0.015878854387248873)  
9   (0.5760016472997092, 0.0068032849792453425)  
10   (0.5975271570923105, 0.007834527963441222)  
12     (0.5776393293559837, 0.0142528263945032)  
15   (0.5453578931771729, 0.017960790510404365)  
20   (0.6776361087819314, 0.012087203246795634)  
[[0.90240165 0.94355872 0.94000119 0.91495218 0.96196082 0.84042467]
 [0.68157424 0.57600165 0.59752716 0.57763933 0.54535789 0.67763611]
 [0.01642271 0.00767356 0.00842437 0.01646376 0.01955943 0.01702722]
 [0.01587885 0.00680328 0.00783453 0.01425283 0.01796079 0.0120872 ]] M7-L0
                

  7%|▋         | 3/45 [00:05<01:21,  1.94s/it]

...

 98%|█████████▊| 44/45 [01:25<00:01,  1.65s/it]

                                             x  \
4     (1.181416081995024, 0.06304851915607394)   
9    (1.1282848293753645, 0.01959217060298397)   
10   (1.0759413445210686, 0.03245950692624889)   
12   (1.1524950597316617, 0.03458375419840756)   
15    (1.0816055131902405, 0.0567063426449035)   
20  (1.6072785135792658, 0.062364993272998145)   

                                             y  
4   (1.0110388442499816, 0.029006362690621634)  
9   (0.9606950462851872, 0.015009188436713259)  
10  (0.9564103421202025, 0.015201199648295692)  
12    (0.9554906965681684, 0.0259408027637678)  
15   (0.9205678932568973, 0.03643228273011058)  
20  (1.0285415957915756, 0.015305327049257102)  
[[1.18141608 1.12828483 1.07594134 1.15249506 1.08160551 1.60727851]
 [1.01103884 0.96069505 0.95641034 0.9554907  0.92056789 1.0285416 ]
 [0.06304852 0.01959217 0.03245951 0.03458375 0.05670634 0.06236499]
 [0.02900636 0.01500919 0.0152012  0.0259408  0.03643228 0.01530533]] M7-L0
                       

100%|██████████| 45/45 [01:27<00:00,  1.66s/it]

                                             x  \
4     (1.1944575501185903, 0.0598035856918894)   
9   (1.0839376463796402, 0.018590683867301873)   
10   (1.029041429414666, 0.031187769023285586)   
12   (1.1011983074143783, 0.03707489403309202)   
15   (0.9956913086125848, 0.06339082463582561)   
20   (1.6531528072383297, 0.06337717845576049)   

                                             y  
4   (1.0110388442499816, 0.029006362690621634)  
9   (0.9606950462851872, 0.015009188436713259)  
10  (0.9564103421202025, 0.015201199648295692)  
12    (0.9554906965681684, 0.0259408027637678)  
15   (0.9205678932568973, 0.03643228273011058)  
20  (1.0285415957915756, 0.015305327049257102)  
[[1.19445755 1.08393765 1.02904143 1.10119831 0.99569131 1.65315281]
 [1.01103884 0.96069505 0.95641034 0.9554907  0.92056789 1.0285416 ]
 [0.05980359 0.01859068 0.03118777 0.03707489 0.06339082 0.06337718]
 [0.02900636 0.01500919 0.0152012  0.0259408  0.03643228 0.01530533]] M7-L0
                       

[index-index space of H_2O-1/J-Cont H_2O-2/H_2O-1,
 index-index space of H_2O-1/J-Cont H-cont/H_2O-1,
 index-index space of H_2O-1/J-Cont CH_4/H_2O-1,
 index-index space of H_2O-1/J-Cont H_2O-2/J-Cont,
 index-index space of H_2O-1/J-Cont H-cont/J-Cont,
 index-index space of H_2O-1/J-Cont CH_4/J-Cont,
 index-index space of H_2O-1/J-Cont H-cont/H_2O-2,
 index-index space of H_2O-1/J-Cont CH_4/H_2O-2,
 index-index space of H_2O-1/J-Cont CH_4/H-Cont,
 index-index space of H_2O-2/H_2O-1 H-cont/H_2O-1,
 index-index space of H_2O-2/H_2O-1 CH_4/H_2O-1,
 index-index space of H_2O-2/H_2O-1 H_2O-2/J-Cont,
 index-index space of H_2O-2/H_2O-1 H-cont/J-Cont,
 index-index space of H_2O-2/H_2O-1 CH_4/J-Cont,
 index-index space of H_2O-2/H_2O-1 H-cont/H_2O-2,
 index-index space of H_2O-2/H_2O-1 CH_4/H_2O-2,
 index-index space of H_2O-2/H_2O-1 CH_4/H-Cont,
 index-index space of H-cont/H_2O-1 CH_4/H_2O-1,
 index-index space of H-cont/H_2O-1 H_2O-2/J-Cont,
 index-index space of H-cont/H_2O-1 H-cont/J-Cont

In [5]:
crts=sel_crt.crts_from_file()

In [6]:
compl=pd.DataFrame([x.completeness for x in crts.values()], index=[x.name for x in crts.values()])
cont=pd.DataFrame([x.contamination for x in crts.values()], index=[x.name for x in crts.values()])

In [7]:
def return_best_index(spt_range):
    return cont[cont[spt_range]<=cont[spt_range].min()+0.002][spt_range]

In [8]:
for x in cont.columns:
    print (return_best_index(x), " \n")

H-cont/H_2O-1 CH_4/J-Cont    0.006656
Name: M7-L0, dtype: float64  

H_2O-1/J-Cont H-cont/H_2O-2    0.000000
H-cont/H_2O-1 CH_4/H_2O-1      0.000000
H-cont/H_2O-1 CH_4/J-Cont      0.000000
H-cont/H_2O-1 CH_4/H-Cont      0.000000
CH_4/H_2O-1 H-cont/J-Cont      0.000000
CH_4/H_2O-1 CH_4/J-Cont        0.000000
CH_4/H_2O-1 CH_4/H-Cont        0.000000
H_2O-2/J-Cont CH_4/J-Cont      0.000000
H_2O-2/J-Cont CH_4/H_2O-2      0.000000
H-cont/J-Cont CH_4/J-Cont      0.001664
H-cont/J-Cont CH_4/H_2O-2      0.001664
H-cont/J-Cont CH_4/H-Cont      0.000000
CH_4/J-Cont H-cont/H_2O-2      0.001664
CH_4/J-Cont CH_4/H_2O-2        0.000000
CH_4/J-Cont CH_4/H-Cont        0.001664
Name: T0-T5, dtype: float64  

H_2O-1/J-Cont H_2O-2/H_2O-1    0.001664
H_2O-1/J-Cont H-cont/H_2O-1    0.001664
H_2O-1/J-Cont CH_4/H_2O-1      0.001664
H_2O-1/J-Cont H_2O-2/J-Cont    0.001664
H_2O-1/J-Cont H-cont/J-Cont    0.001664
H_2O-1/J-Cont CH_4/J-Cont      0.001664
H_2O-1/J-Cont H-cont/H_2O-2    0.001664
H_2O-1/J-Cont CH_4/H

In [9]:
to_use={'H_2O-1/J-Cont CH_4/H-Cont':['L0-L5'],
        'H_2O-2/H_2O-1 H_2O-2/J-Cont':['L5-T0'],
        #'H_2O-1/J-Cont H-cont/H_2O-2':['M7-L0'],
        'H-cont/H_2O-1 H_2O-2/J-Cont':['T0-T5'],
        'H_2O-2/H_2O-1 CH_4/H-Cont':['Y dwarfs'],
        'H-cont/J-Cont CH_4/H-Cont':['T5-T9'],
        'CH_4/H_2O-1 H-cont/J-Cont':['subdwarfs']}
        

In [10]:
good_indices=[crts[k] for k in to_use.keys()]

In [11]:
good_indices

[index-index space of H_2O-1/J-Cont CH_4/H-Cont,
 index-index space of H_2O-2/H_2O-1 H_2O-2/J-Cont,
 index-index space of H-cont/H_2O-1 H_2O-2/J-Cont,
 index-index space of H_2O-2/H_2O-1 CH_4/H-Cont,
 index-index space of H-cont/J-Cont CH_4/H-Cont,
 index-index space of CH_4/H_2O-1 H-cont/J-Cont]

In [12]:
def box_parameters(idx, spt_range):
    bs=idx.shapes
    b=[x for x in bs if x.shape_name==spt_range][0]
    print ('{}  {} m: {} b: {} s:{}, comp : {}, cont: {}'.format(spt_range, idx,  round(b.coeffs[0], 2), round(b.coeffs[1], 2), round(b.scatter, 2), round(idx.completeness[spt_range], 2),  round(idx.contamination[spt_range], 3)))

In [13]:
for idx, k in zip(good_indices, to_use.keys()):
    spt_range=to_use[k][0]
    box_parameters(idx, spt_range)

IndexError: list index out of range

In [ ]:
good_indices[0]

In [ ]:
import matplotlib as mpl
mpl.rcParams['legend.fontsize'] = 'small'


In [ ]:
fig, ax=plt.subplots(ncols=3, nrows=2, figsize=(12, 8))
good_indices[0].plot(ax=ax[0][0], highlight=to_use[good_indices[0].name], xlim=[0.0, 2.], ylim=[.5, 1.5])
good_indices[1].plot(ax=ax[0][1],  highlight=to_use[good_indices[1].name], xlim=[0.0, 1.2], ylim=[0.0, .8])
good_indices[2].plot(ax=ax[0][2],  highlight=to_use[good_indices[2].name], xlim=[1.0,3.], ylim=[-.5, .5])


good_indices[3].plot(ax=ax[1][0],  highlight=to_use[good_indices[3].name], xlim=[-10, 10.0], ylim=[-.25, .3])
good_indices[4].plot(ax=ax[1][1],  highlight=to_use[good_indices[4].name], xlim=[.3, 1.2], ylim=[-.5, 1.5])
good_indices[5].plot(ax=ax[1][2],  highlight=to_use[good_indices[5].name], xlim=[0.2, 1.2], ylim=[0.4, 0.8])


ax[0][0].set_title(to_use[good_indices[0].name][0])
ax[0][1].set_title(to_use[good_indices[1].name][0])
ax[0][2].set_title(to_use[good_indices[2].name][0])

ax[1][0].set_title(to_use[good_indices[3].name][0])
ax[1][1].set_title(to_use[good_indices[4].name][0])
ax[1][2].set_title(to_use[good_indices[5].name][0])

for a in np.concatenate(ax)[1:]:
    a.legend().set_visible(False)
plt.minorticks_on()
plt.tight_layout()
plt.savefig(wisps.OUTPUT_FIGURES+'/index_index_plots.pdf', bbox_inches='tight', dpi=200)

### Use these good indices to select objects, ignore f-test

In [ ]:
dft=dt.rename(columns={'grism_id':'Names'})

In [ ]:
import itertools

In [ ]:
cands=[]
for idx, k in zip(good_indices, to_use.keys()):
    spt_range=to_use[k][0]
    print (spt_range)
    bs=idx.shapes
    bx=[x for x in bs if x.shape_name==spt_range][0]
    df_to_use=wisps.Annotator.reformat_table(dft[[idx.xkey, idx.ykey]])
    selec_indx_array=bx.select(df_to_use).index
    ls=dft.iloc[selec_indx_array].Names
    print (len(np.unique(ls))/len(dft))
    cands.append(ls)

In [ ]:
merged = np.array(list(itertools.chain(*cands)))

In [ ]:
final_cands=np.unique(merged.flatten())

In [ ]:
dt[dt.grism_id.isin(final_cands)].to_pickle(wisps.OUTPUT_FILES+'/selected_by_indices.pkl')

In [ ]:
len(dt[dt.grism_id.isin(final_cands)])

In [ ]:
import seaborn as sns 
cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9, as_cmap=True)

In [ ]:
sel_crt.plot_cont_compl(cmap=cmap)

In [ ]:
crt_vals=crts.values()
conts=pd.DataFrame([ x.contamination for x in crt_vals]).applymap( lambda x: round(x, 2))
compls=pd.DataFrame([ x.completeness for x in crt_vals]).applymap(lambda x: round(x, 2))
conts['index-space']=[x.name for x in crt_vals]
compls['index-space']=[x.name for x in crt_vals]

In [ ]:
conts.to_latex(wisps.LIBRARIES+'/completeness.tex', index=False, na_rep=' ')
conts.to_latex(wisps.LIBRARIES+'/contamination.tex', index=False, na_rep=' ')

In [ ]:
#save the indices used here

In [ ]:
import pickle
#save the random forest
output_file=wisps.OUTPUT_FILES+'/best_indices_to_use.pkl'
with open(output_file, 'wb') as file:
    pickle.dump(to_use,file)

In [ ]:
df=wisps.datasets['candidates']

In [ ]:
df.f_test.plot(kind='hist')

In [ ]:
wisps